In [1]:
from gensim.models import FastText
import os,sys
from urllib.parse import urlparse
from getImagesFunctions import *
from getDataFunctions import *
from gensim.models import Word2Vec
import gensim
import random
import langid
import re

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
urls = gatherPagesUrlsFolder("C:/Users/Ruben/Documents/GitHub/ReACT_GCV/code/work/scrape_environment/image_npg_2")

corrupted json file, probably an 400 Error!
corrupted json file, probably an 400 Error!
corrupted json file, probably an 400 Error!
finished gatherPagesUrlsFolder


In [3]:
urlsp = [[x for x in urlparse(i) if i][2:] for i in urls]
urlsp = [[x for x in i if x] for i in urlsp]
urlsp = [" ".join(x) for x in urlsp]

In [4]:
delimiters = "/", ".", " "
regexPattern = '|'.join(map(re.escape, delimiters))

In [5]:
sentences = []
suitable = 0
unsuitable = 0
for u in urlsp:
    u = re.split(regexPattern, u)
    u = [x for x in u if x]
    if len(u) == 0:
        unsuitable += 1
        continue
    maxu = max(u, key=len)
    if "-" in maxu:
        sentences.append(maxu.split('-'))
        suitable += 1
    else:
        unsuitable += 1
        
print("found {} suitable and {} unsuitable links. {}% of the links are suitable".format(suitable, unsuitable, round(suitable/len(urlsp)*100)))


found 37094 suitable and 7764 unsuitable links. 83% of the links are suitable


In [6]:
model = gensim.models.Word2Vec(sentences, size=100, window=25, min_count=1, workers=6,sg=1)
model.save("model_paths.w2v")
mp = gensim.models.KeyedVectors.load('model_paths.w2v')

In [15]:
def DistanceMatrix(sentences, model):
    total_list = list()
    
    for word in sentences:
        
        list_sentences = list()
        
        for term in list_sentences:
            #tmp = model.distance(word, term)
            tmp = model.wmdistance(" ".join(word), " ".join(word))

            list_sentences.append(tmp)
        
        total_list.append(list_sentences)
    df = pd.DataFrame(total_list, columns = list_sentences, index = list_sentences)
    return df

def ClusterWord(matrix, k):
    centroids,_ = kmeans(matrix,k)
    idx,_ = vq(matrix,centroids)
    
    return dict(zip(list(matrix.index), idx))

def GetNumClus(distance_matrix):
    af = AffinityPropagation(affinity='precomputed',preference=-2).fit(np.array(distance_matrix.values))
    cluster_centers_indices = af.cluster_centers_indices_
    labels = af.labels_

    no_clusters = len(cluster_centers_indices)
    return no_clusters

In [18]:
DistanceMatrix(sentences[0:1000],mp)

created list with 1000 words


""


In [17]:
sentences[0]

['girl',
 'photo',
 'vietnam',
 'war',
 'kim',
 'phuc',
 'laser',
 'treatment',
 'napalm',
 'burns',
 'video']